# Uniswap V3 Subgraph Liquidity Positions Example

This notebook is just the `.ipynb` (for easier reading of both code and output on github) implementation of [this code](https://github.com/atiselsts/uniswap-v3-liquidity-math/blob/master/subgraph-liquidity-positions-example.py) (credit: https://github.com/atiselsts) with slight modifications, showing an example on how to query the Uniswap v3 Subgraph to show all active positions in the USDC/WETH 0.3% pool.

See the technical note [Liquidity Math in Uniswap v3](http://atiselsts.github.io/pdfs/uniswap-v3-liquidity-math.pdf), [Uniswap v3 whitepaper](https://uniswap.org/whitepaper-v3.pdf), and [Uniswap V3 Subgraph](https://thegraph.com/legacy-explorer/subgraph/uniswap/uniswap-v3) for more details.

**Library dependency**: The code uses `gql` to query the Uniswap V3 Subgraph API.

In [ ]:
# Example that shows all active positions in the 0.3% USDC/ETH
# pool using data from the Uniswap v3 subgraph.

from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import math

# 0.3% USDC/ETH pool
POOL_ID = "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8"

# If you want to test with a smaller pool:
# 0.05% GUSD/DAI pool (small pool)
#POOL_ID = "0x7cf12cef5ce9e5e068ebdef470ff8295e26c47b9"

URL = "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3"

pool_query = """query get_pools($pool_id: ID!) {
  pools(where: {id: $pool_id}) {
    tick
    sqrtPrice
    liquidity
    feeTier
    token0 {
      symbol
      decimals
    }
    token1 {
      symbol
      decimals
    }
  }
}"""

# return open positions only (with liquidity > 0)
position_query = """query get_positions($num_skip: Int, $pool_id: ID!) {
  positions(skip: $num_skip, where: {pool: $pool_id, liquidity_gt: 0}) {
    id
    tickLower { tickIdx }
    tickUpper { tickIdx }
    liquidity
  }
}"""

block_query = """query {
  _meta {
    block {
      number
    }
  }
}"""

In [ ]:
TICK_BASE = 1.0001

def tick_to_price(tick):
    return TICK_BASE ** tick

In [ ]:
client = Client(
    transport=RequestsHTTPTransport(
        url=URL,
        verify=True,
        retries=5,
    ))

In [ ]:
# get position info
positions = []
num_skip = 0
try:
    while True:
        #print("Querying positions, num_skip={}".format(num_skip))
        variables = {"num_skip": num_skip, "pool_id": POOL_ID}
        response = client.execute(gql(position_query), variable_values=variables)

        if len(response["positions"]) == 0:
            break
        num_skip += len(response["positions"])
        for item in response["positions"]:
            tick_lower = int(item["tickLower"]["tickIdx"])
            tick_upper = int(item["tickUpper"]["tickIdx"])
            liquidity = int(item["liquidity"])
            id = int(item["id"])
            positions.append((tick_lower, tick_upper, liquidity, id))
except Exception as ex:
    print("got exception while querying position data:", ex)
    exit(-1)

print("Total number of positions found = {}".format(len(positions)))

Total number of positions found = 3552


In [ ]:
try:
    variables = {"pool_id": POOL_ID}
    response = client.execute(gql(pool_query), variable_values=variables)

    if len(response['pools']) == 0:
        print("pool not found")
        exit(-1)

    pool = response['pools'][0]
    pool_liquidity = int(pool["liquidity"])
    current_tick = int(pool["tick"])

    token0 = pool["token0"]["symbol"]
    token1 = pool["token1"]["symbol"]
    decimals0 = int(pool["token0"]["decimals"])
    decimals1 = int(pool["token1"]["decimals"])
except Exception as ex:
    print("got exception while querying pool data:", ex)
    exit(-1)

In [ ]:
# get block height, for record
try:
    response = client.execute(gql(block_query))
    blocknum = response['_meta']['block']['number']
    print("Ethereum block height {}".format(blocknum))
except Exception as ex:
    print("got exception", ex)

Ethereum block height 13339848


In [ ]:
# Compute and print the current price
current_price = tick_to_price(current_tick)
current_sqrt_price = tick_to_price(current_tick / 2)
adjusted_current_price = current_price / (10 ** (decimals1 - decimals0))
print("Current price={:.6g} {} for {} at tick {}".format(adjusted_current_price, token1, token0, current_tick))

# Sum up all the active liquidity and total amounts in the pool
active_positions_liquidity = 0
total_amount0 = 0
total_amount1 = 0

# Print all active positions
for tick_lower, tick_upper, liquidity, id in sorted(positions):

    sa = tick_to_price(tick_lower / 2)
    sb = tick_to_price(tick_upper / 2)

    if tick_upper < current_tick:
        # Only token1 locked
        amount1 = liquidity * (sb - sa) # eq(9) in technical note
        total_amount1 += amount1

    elif tick_lower <= current_tick <= tick_upper:
        # Both tokens present
        amount0 = liquidity * (sb - current_sqrt_price) / (current_sqrt_price * sb) # eq(12) in technical note
        amount1 = liquidity * (current_sqrt_price - sa) # eq(13) in technical note
        adjusted_amount0 = amount0 / (10 ** decimals0)
        adjusted_amount1 = amount1 / (10 ** decimals1)

        total_amount0 += amount0
        total_amount1 += amount1
        active_positions_liquidity += liquidity

        print("  position {: 7g} in range [{},{}]: {:.6g} {} and {:.6g} {} at the current price".format(
              id, tick_lower, tick_upper,
              adjusted_amount0, token0, adjusted_amount1, token1))
    else:
        # Only token0 locked
        amount0 = liquidity * (sb - sa) / (sa * sb) # eq(4) in technical note
        total_amount0 += amount0


print("In total (including inactive positions): {:.6g} {} and {:.6g} {}".format(
      total_amount0 / 10 ** decimals0, token0, total_amount1 / 10 ** decimals1, token1))
print("Total liquidity from active positions: {}, from pool: {} (should be equal...)".format(
      active_positions_liquidity, pool_liquidity))

Current price=0.000303297 WETH for USDC at tick 195312
  position  104211 in range [-887220,887220]: 1.13261 USDC and 0.000343517 WETH at the current price
  position  127449 in range [-887220,887220]: 4.79148 USDC and 0.00145324 WETH at the current price
  position  104069 in range [-887220,887220]: 5.75536 USDC and 0.00174558 WETH at the current price
  position  114656 in range [-887220,887220]: 7.9392 USDC and 0.00240794 WETH at the current price
  position  110416 in range [-887220,887220]: 252.6 USDC and 0.076613 WETH at the current price
  position  128009 in range [-887220,887220]: 339.887 USDC and 0.103087 WETH at the current price
  position  106613 in range [-887220,887220]: 443.445 USDC and 0.134496 WETH at the current price
  position  119305 in range [-887220,887220]: 1015.94 USDC and 0.308133 WETH at the current price
  position  127637 in range [-887220,887220]: 1021.31 USDC and 0.309759 WETH at the current price
  position  116530 in range [-887220,887220]: 3475.65 USD

  position  133791 in range [194700,198480]: 165250 USDC and 10.3107 WETH at the current price
  position   33039 in range [194700,198780]: 11319.8 USDC and 0.649921 WETH at the current price
  position  107166 in range [194700,198900]: 49.9746 USDC and 0.00278139 WETH at the current price
  position   25422 in range [194700,198900]: 92368.6 USDC and 5.14087 WETH at the current price
  position   39420 in range [194700,199320]: 786.32 USDC and 0.0395778 WETH at the current price
  position   29572 in range [194700,199380]: 756.382 USDC and 0.0375638 WETH at the current price
  position   59029 in range [194700,200280]: 929.666 USDC and 0.038633 WETH at the current price
  position   34164 in range [194700,200280]: 13763.8 USDC and 0.571965 WETH at the current price
  position   30114 in range [194700,200340]: 364.119 USDC and 0.0149722 WETH at the current price
  position   36944 in range [194700,200340]: 783.239 USDC and 0.032206 WETH at the current price
  position   44113 in range [